In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col

In [ ]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
            .appName('SparkDataStreaming') \
            .config('spark.jars.packages', "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0,"
                                           "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2") \
            .config('spark.cassandra.connection.host', 'cassandra') \
            .getOrCreate()

In [ ]:
# Đọc dữ liệu từ Kafka topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:29092,kafka2:29092,kafka3:29092") \
  .option("subscribe", "users_created") \
  .option("startingOffsets", "earliest") \
  .load()

In [ ]:
print(df)

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['cassandra'])
session = cluster.connect()

In [ ]:
from pyspark.sql.types import *

schema = StructType([
        StructField("id", StringType(), False),
        StructField("first_name", StringType(), False),
        StructField("last_name", StringType(), False),
        StructField("gender", StringType(), False),
        StructField("address", StringType(), False),
        StructField("post_code", StringType(), False),
        StructField("email", StringType(), False),
        StructField("username", StringType(), False),
        StructField("registered_date", StringType(), False),
        StructField("phone", StringType(), False),
        StructField("picture", StringType(), False)
    ])

sel = df.selectExpr("CAST(value AS STRING)") \
        .select(from_json(col('value'), schema).alias('data')).select("data.*")
print(sel)

In [ ]:
# #export result
# query = sel \
#     .writeStream \
#     .outputMode("append") \
#     .format("csv") \
#     .option("header", "true") \
#     .option("checkpointLocation", "work") \
#     .option("path", "work/da.csv") \
#     .start()

# query.awaitTermination()


In [ ]:
streaming_query = (sel.writeStream.format("org.apache.spark.sql.cassandra")
                               .option('checkpointLocation', '/tmp/checkpoint')
                               .options(keyspace='spark_streams', table='created_users')
                               .start())

In [ ]:
streaming_query.awaitTermination()